In [1]:
#pip install NRCLex

In [2]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
# Import earning calls
tickers = ["AAPL","MMM"]

texts = []
list_ids = []
Ticker = []
number_texts_per_stock = []
for ticker in tickers:
    dir_path = "data/text/"+ticker+"/"
    list_ids1 = os.listdir(dir_path)
    ticker1 = os.path.basename(os.path.dirname(dir_path))
    ticker1 = ticker1.split()
    texts1 = []
    for s in list_ids1:
        with open(dir_path + s, encoding = 'ISO-8859-1') as f:
            t = f.read()
            texts1.append(t)
    texts = texts + texts1
    list_ids = list_ids + list_ids1 
    Ticker = Ticker + ticker1*len(list_ids1)
    number_texts_per_stock.append(len(list_ids1))
print(len(texts))
print(len(list_ids))
print(len(Ticker))
print(number_texts_per_stock)

In [ ]:
# Get dates
data_path = "data/ticker/id_to_date"
with open(data_path,"rb") as file :
    id_to_date = pickle.load(file)

In [ ]:
# Create dataframe with earning calls informations
df1 = pd.DataFrame(data={'id': list_ids})
df1['date']=list_ids
df1=df1.replace({'date': id_to_date})
df1['date'] = pd.to_datetime(df1['date'])
df1['quarter'] = df1['date'].dt.to_period('Q')
df1['ticker']= Ticker
df1=df1.replace({'ticker': id_to_date})
df1= df1.sort_values(['ticker','date'], ascending=True)
display(df1)

In [ ]:
## To do : keep same sumber of raws for each stock
## For the moment : only 2 stocks of same lenghth so ok!

# Import datas from yahoo finance

Import html tickers from data/ticker
dir_path = "data/ticker/"
list_tickers = os.listdir(dir_path)
list_tickers.remove('.DS_Store') # for mac
list_tickers.remove('id_to_date')
print(list_tickers)

Test with few stocks
list_tickers = ['AAPL']

In [ ]:
# Get the month from which we import
# Q1 : from january (1) to march (3)
# Q2 : from april (4) to june (6)
# Q3 : from july (7) to septmber (9)
# Q4 : from october (10) to december (12)

quarter_start = df1['quarter'].iloc[0].quarter
month_start = quarter_start*2-1
quarter_end = df1['quarter'].iloc[-2].quarter # take the last quarter
month_end = quarter_end*3

if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30

In [ ]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

start = datetime.datetime(df1['date'].iloc[0].year,month_start,1)
end   = datetime.datetime(df1['date'].iloc[-1].year,month_end,day_end)

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
list_tickers = list(set(Ticker))
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
stock_final['date']=stock_final.index
stock_final['quarter'] = stock_final['date'].dt.to_period('Q')
stock_final['market_cap']=stock_final['Adj Close']*stock_final['Volume']
stock_final=stock_final.drop(columns=['Open', 'High','Low','Close','date','Adj Close','Volume'])
display(stock_final)

In [ ]:
stock_final = stock_final.groupby(['ticker','quarter']).sum()
stock_final['percent_change']=stock_final['market_cap'].pct_change()
display(stock_final)
Quarter = stock_final.index.get_level_values(1)

# Text analysis

In [ ]:
# Preprocessing
n= len(texts)
texts = [processing(x) for x in texts]

In [ ]:
#N=len(texts)

for i in tqdm(range(n)):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

In [ ]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)
df[column]=df[column].astype(float)

#number_words = np.zeros(n)
#for i in range(n):
#    number_words[i] = len(texts[i])
#df['number_words']=number_words
df['id'] = list_ids
df['date']=list_ids
df=df.replace({'date': id_to_date})
df['date'] = pd.to_datetime(df['date'])
#df['quarter'] = Quarter
df['ticker']= Ticker
df= df.sort_values(by=['ticker','date'], ascending=True)
# Remove most recent text for each ticker
df=df.drop(df.groupby('ticker').tail(1).index, axis=0)
# Add quarters
df['quarter']=Quarter
df_final=df
display(df)

In [ ]:
df_change = df_final
column_change=['anger_change','anticipation_change','disgust_change','fear_change','joy_change','negative_change','positive_change','sadness_change','surprise_change','trust_change']
df_change[column] = df[column].pct_change()
#df_change.columns =column_change
df_change = df_change.drop(columns=['id','date'])
df_change = df_change.groupby(['ticker','quarter']).sum()
display(df_change)

In [ ]:
# Merge dataframes
df_change = pd.merge(stock_final, df_change, left_index=True, right_index=True)
# Drop first row
df_change=df_change.drop(df_change.groupby('ticker').head(1).index, axis=0)
display(df_change)

In [ ]:
# ols
result = sm.ols(formula="percent_change ~ anger+anticipation+fear+joy+negative+positive+sadness+surprise+trust", data=df_change).fit()
#anger+anticipation+disgust+fear+joy+negative+positive+sadness+surprise+trust
# ATTENTION NAN DANS DiSGUST
result.summary()

# In number of words ==> better to keep frequency

df[column]=df[column].multiply(df["number_words"], axis = "index")
#df[["A", "B"]].multiply(df["C"], axis="index")
display(df)

In [ ]:
df_mean = pd.DataFrame(df[column].mean())
df_mean.columns = ['Average frequency']
ax = df_mean.plot.bar(stacked=True)

# Stock returns

# A faire pour cette méthode : 
- classer les earning calls par quarter et par stock index
- importer les stock returns et garder seulement ceux des earning calls (mapping permno/ stock index)

# liens utiles

https://betterprogramming.pub/unlocking-emotions-in-text-using-python-6d062b48d71f